In [1]:
%load_ext autoreload
%autoreload 2

import helpers

In [2]:
import os
from sklearn.externals import joblib
import pandas as pd
import numpy as np
import random
import json
import datetime
import math

In [3]:
%run "./../app_webscr_otodom/scraper/settings.py"

In [4]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'helpers', '_i2', 'os', 'joblib', 'pd', 'np', 'random', 'json', 'datetime', 'math', '_i3', '__nonzero__', 'BOT_NAME', 'MAX_PAGES', 'XPATH_JSON_DIR', 'SAVE_RESULTS', 'LOCAL_DATA_PATH', 'LOCAL_DATA_PATH_PREP', 'LOCAL_DATA_PATH_DICT', 'BUCKET_NAME', 'BUCKET_PREFIX_BSON', 'MONGO_ADDRESS', 'MONGO_PORT', 'MONGO_DBNAME', 'MONGO_COLL_OTODOM', 'MONGO_USERNAME', 'MONGO_PASSWORD', 'DOWNLOAD_IMAGES', 'SPIDER_MODULES', 'NEWSPIDER_MODULE', 'LOG_ENABLE', 'LOG_LEVEL', 'ROBOTSTXT_OBEY', 'DOWNLOAD_DELAY', 'RANDOMIZE_DOWNLOAD_DELAY', 'CONCURRENT_REQUESTS', 'USER_AGENT', 'COOKIES_ENABLED', '_i4'])

In [5]:
db = helpers.FilesMongo.set_connection(MONGO_ADDRESS, MONGO_PORT, MONGO_DBNAME, MONGO_COLL_OTODOM, MONGO_USERNAME, MONGO_PASSWORD)


In [6]:
%%time
clf = joblib.load('model.pkl')

CPU times: user 6.21 s, sys: 547 ms, total: 6.76 s
Wall time: 7.98 s


In [7]:
offers = db.find({ "prediction_time" : { "$exists" : False } })

In [8]:
def write_prediction(raw_data):
    data = helpers.Scraper.dict_except(raw_data, except_keys=['gallery','img_gallery_strimg'])
    df = pd.DataFrame([data])
    np.random.seed(666)
    random.seed(666)
    os.environ['PYTHONHASHSEED'] = str(666)
    prediction = clf.predict(df)
    prediction = np.expm1(prediction)
    prediction = int(pd.to_numeric(prediction[0], errors='coerce'))
    price = int(pd.to_numeric(raw_data['price'], errors='coerce')) if not(math.isnan(raw_data['price'])) else None
    
    try:
        wyn  = (raw_data["_id"], db.update_one(
            {"_id" : raw_data['_id']},
            {"$set":{"prediction": prediction, 
                     "price_model_dif": price-prediction if (price is not None and prediction is not None) else None,
                     "prediction_time": datetime.datetime.now()}
            }))
    except Exception as e:
        print(e)
        
        
    
     


In [9]:
for offer in offers:
    write_prediction(offer)